#### Imports

In [1]:
import json
import nltk
from nltk.collocations import *
import datetime as dt
import locale
import spacy
from tqdm import tqdm
import pprint
import pandas as pd
from ast import literal_eval
import os
import gensim
import gensim.corpora as corpora
from pprint import pprint


In [ ]:
!python -m spacy download de_core_news_lg

In [ ]:
#NOTE: default version doesntwork in colab
!pip install pyLDAvis==2.1.2

#### Read Data

In [4]:
# connect with google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
#change cwd
%cd drive/MyDrive/Work/Frontline/data

/content/drive/.shortcut-targets-by-id/1WfnZsqpG1r110J63sMbfS5TpsDOkveiV/data


In [6]:
# paths
FILTERED_PATH="filtered_4_26"

In [7]:
## FOR NOW: USE SMALL DATA SET
df=pd.read_csv("sample.csv")

### Topic Modelling


#### Prepare Data


In [8]:
# custom module
import preprocessing

In [9]:
# Load model
spacy_mod = spacy.load("de_core_news_lg", disable=['ner', 'parser', 'tagger'])

In [10]:
# read custom stopwords

# open list of custom stopwords
custom_stop_words= open("custom_stopwords.txt").read().split()

# add custom stopwords to model
for word in custom_stop_words:
  spacy_mod.Defaults.stop_words.add(word)

In [11]:
# convert corpus to language object
spacy_lang = []
for i, doc in tqdm(df.iterrows()): 
  spacy_lang.append(spacy_mod("".join(doc['text'])))

50it [00:02, 22.20it/s]


In [12]:
# preprocess: remove stopwords
spacy_cleaned = []
for doc in tqdm(spacy_lang): 
    spacy_cleaned.append(preprocessing.preprocess(doc, remove_ent=True))

100%|██████████| 50/50 [00:00<00:00, 1732.73it/s]


#### Topic Analysis

In [13]:
import pyLDAvis
import pyLDAvis.gensim
import pickle 

#### Creating the model

In [30]:
# Create Dictionary
id2word = corpora.Dictionary(spacy_cleaned)

# Create Corpus
texts = spacy_cleaned

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [29]:
# number of topics
num_topics = 5
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,id2word=id2word, num_topics=num_topics)
# Print the key words for each topic
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[(0,
  '0.009*"frauen" + 0.009*"frau" + 0.007*"gewalt" + 0.004*"häuslicher" + '
  '0.004*"opfer" + 0.004*"mann" + 0.003*"polizei" + 0.003*"männer" + '
  '0.002*"kinder" + 0.002*"menschen"'),
 (1,
  '0.010*"frauen" + 0.008*"gewalt" + 0.006*"opfer" + 0.006*"frau" + '
  '0.005*"mann" + 0.004*"häuslicher" + 0.003*"männer" + 0.003*"leben" + '
  '0.003*"landkreis" + 0.002*"menschen"'),
 (2,
  '0.011*"frau" + 0.009*"gewalt" + 0.007*"mann" + 0.007*"frauen" + '
  '0.006*"opfer" + 0.005*"polizei" + 0.003*"häuslicher" + 0.003*"männer" + '
  '0.003*"leben" + 0.002*"fälle"'),
 (3,
  '0.016*"gewalt" + 0.011*"frauen" + 0.008*"opfer" + 0.006*"frau" + '
  '0.005*"häuslicher" + 0.004*"mann" + 0.003*"männer" + 0.003*"prozent" + '
  '0.003*"polizei" + 0.003*"leben"'),
 (4,
  '0.019*"gewalt" + 0.018*"frauen" + 0.007*"opfer" + 0.007*"frau" + '
  '0.005*"häuslicher" + 0.005*"mann" + 0.004*"männer" + 0.004*"polizei" + '
  '0.004*"prozent" + 0.003*"fälle"')]


In [34]:
# saving model
lda_model.save("models/tmodel_sample")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


#### Loading a previously saved model

In [36]:
lda_model=gensim.models.LdaMulticore.load("models/tmodel_sample", mmap="r")
lda_model.print_topics()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[(0,
  '0.009*"frauen" + 0.009*"frau" + 0.007*"gewalt" + 0.004*"häuslicher" + 0.004*"opfer" + 0.004*"mann" + 0.003*"polizei" + 0.003*"männer" + 0.002*"kinder" + 0.002*"menschen"'),
 (1,
  '0.010*"frauen" + 0.008*"gewalt" + 0.006*"opfer" + 0.006*"frau" + 0.005*"mann" + 0.004*"häuslicher" + 0.003*"männer" + 0.003*"leben" + 0.003*"landkreis" + 0.002*"menschen"'),
 (2,
  '0.011*"frau" + 0.009*"gewalt" + 0.007*"mann" + 0.007*"frauen" + 0.006*"opfer" + 0.005*"polizei" + 0.003*"häuslicher" + 0.003*"männer" + 0.003*"leben" + 0.002*"fälle"'),
 (3,
  '0.016*"gewalt" + 0.011*"frauen" + 0.008*"opfer" + 0.006*"frau" + 0.005*"häuslicher" + 0.004*"mann" + 0.003*"männer" + 0.003*"prozent" + 0.003*"polizei" + 0.003*"leben"'),
 (4,
  '0.019*"gewalt" + 0.018*"frauen" + 0.007*"opfer" + 0.007*"frau" + 0.005*"häuslicher" + 0.005*"mann" + 0.004*"männer" + 0.004*"polizei" + 0.004*"prozent" + 0.003*"fälle"')]

In [40]:
# Create Corpus
texts = spacy_cleaned

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# Saving dictionary of imported model
id2word=lda_model.id2word

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Vizualizing models

In [41]:
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(lda_model, corpus, id2word)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/pyLDAvis/_prepare.py:232: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.021167  0.000943       1        1  46.117990
2      0.018841 -0.010127       2        1  16.663563
1     -0.000267 -0.001187       3        1  13.905158
3     -0.008362 -0.005013       4        1  11.945833
0      0.010956  0.015383       5        1  11.367456, topic_info=          Term       Freq      Total Category  logprob  loglift
8         frau  53.000000  53.000000  Default  30.0000  30.0000
114     gewalt  99.000000  99.000000  Default  29.0000  29.0000
176      opfer  47.000000  47.000000  Default  28.0000  28.0000
332     frauen  92.000000  92.000000  Default  27.0000  27.0000
15        mann  34.000000  34.000000  Default  26.0000  26.0000
..         ...        ...        ...      ...      ...      ...
819    partner   1.123978  11.212850   Topic5  -6.5713  -0.1258
191       ring   1.073520  10.099886   Topic5  -6.6173  -0.0672
844     zahlen   1.057799  10.117484   Topic5  -6.6320  -0.0837
771    getötet   1.058789  12.861991   Topic5  -6.6311  -0.3227
663  landkreis   1.036384  14.814065   Topic5  -6.6525  -0.4854

[381 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
0         1  0.343510        abend
0         2  0.343510        abend
0         4  0.343510        abend
1792      2  0.832285  absichtlich
32        1  0.415919         alte
...     ...       ...          ...
923       1  0.397636   übergriffe
923       2  0.198818   übergriffe
923       3  0.198818   übergriffe
923       5  0.198818   übergriffe
924       5  0.951270  überlebende

[500 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 3, 2, 4, 1])

END OF CODE